In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [5]:
transf = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data',train=True, download=True, transform=transf)
trainloader = DataLoader(trainset, batch_size=8, shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data',train=False, download=True, transform=transf)
testloader = DataLoader(testset, batch_size=8, shuffle=False, num_workers=2)
classes = ('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')

Files already downloaded and verified
Files already downloaded and verified


# Build a model

In [9]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module): # 내가 사용하고자 하는 연산 정의
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3,6,5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)

    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x))) # conv1->relu->MaxPooling
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1,16*5*5)               # 사이즈 변형(일차로 변환)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()  # instance하나 만들기

In [10]:
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


# Implement the model with training set

In [11]:
import torch.optim as optim 

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9) # net안에 있는 parameter upgrade

# 지도학습 하
# output이 나와있고 output을 바탕으로 loss 구하기
# loss바탕으로 gradient
# 다시 parameter upgrade
# gradient 구하기 위한 optimizer

In [13]:
for epoch in range(1):
    running_loss = 0.0
    for i, data in enumerate(trainloader,0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        
        # zero the parameter gradients(옵티마이저 초기화)
        optimizer.zero_grad() 
        
        # update 과정(8개 이미지 가지고, batch_size=8)
        # forward + backward + optimize
        outputs = net(inputs) # image를 모델에 넣기
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step() # optimizer 계산한다

        # print statistics
        running_loss+=loss.item()
        if i % 2000 == 1999:   # print every 2000 mini-batches
            print('[%d, %5d] loss:%.3f' %
            (epoch + 1, i+1, running_loss/2000))
            running_loss = 0.0
print('Finished Training')

[1,  2000] loss:2.177
[1,  4000] loss:1.849
[1,  6000] loss:1.637
Finished Training


# save the trained model

In [14]:
PATH = './cifar_net.path'
torch.save(net.state_dict(),PATH)

# Load the pre_trained model

In [15]:
net = Net()
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

# 예측

In [18]:
correct = 0
total = 0
with torch.no_grad(): # gradient update안한다
    for data in testloader:
        images, labes = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data,1) # np.argmax랑 같다
        total+=labels.size(0)
rrect = 0
total = 0
with torch.no_grad(): # gradient update안한다
    for data in testloader:
        images, labes = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data,1) # np.argmax랑 같다
        total+=labels.size(0)
        correct+=(predicted==labels).sum().item() # predict이랑 label이 같으면 correct개수 추가
print('Accuracy of the network on the 10000 test images: %d %%' %(
    100* correct/total
))

Accuracy of the network on the 10000 test images: 8 %
